In [16]:
import numpy as np
from Bio import SeqIO

np.random.seed(0)

nucleotides = ['A', 'T', 'G', 'C']

sequence_string = ''.join(np.random.choice(nucleotides, 1000).tolist())
reads = ''

for i in range(1, 1001):
    read_len = 150
    read_start = np.random.default_rng().integers(0, 1000 - read_len)
    read_str = sequence_string[read_start:read_start+read_len]
    reads += f'>READ_SEQ_{i}\n{read_str}\n'

with open(f"reads.fasta", "w") as file:
    file.write(reads)

with open("sequense.fasta", "w") as file:
    file.write('>MAIN_SEQ_0\n')
    file.write(sequence_string)

## Построение графа Де Брюина
По заданному набору ридов в формате FASTQ и параметру k, который соответствует длине k-меров, построить граф Де Брюина, некоторый путь в котором соответствовал бы возможной подстроке в исходном геноме. Не забывайте про запоминание покрытия каждого k-мера, а так же про сами подстроки, которые соответствуют каждому ребру. В остальном граф полностью соответствует тому, что был описан в лекции.

In [16]:
import numpy as np
from Bio import SeqIO

seq_name, seq_string = [], []
with open ("/home/pk/Desktop/BOTAY/BioinformaticsCourse2024PK/homework/2_2/reads.fasta",'r') as fa:
    for seq_record  in SeqIO.parse(fa,'fasta'):
        seq_name.append(str(seq_record.name))
        seq_string.append(str(seq_record.seq))

true_seq_name, true_seq_string = [], []
with open ("/home/pk/Desktop/BOTAY/BioinformaticsCourse2024PK/homework/2_2/sequense.fasta",'r') as fa:
    for seq_record  in SeqIO.parse(fa,'fasta'):
        true_seq_name.append(str(seq_record.name))
        true_seq_string.append(str(seq_record.seq))
true_seq = true_seq_string[0]

In [27]:
from collections import defaultdict
from copy import deepcopy


class Node():
    def __init__(self, string):
        self.string = string
        self.in_degree = 0
        self.out_degree = 0

class Edge():
    def __init__(self, prev_node, next_node, string):
        self.prev_node = prev_node
        self.next_node = next_node
        self.string = string
        self.coverage = 1

class DBGraph():
    def __init__(self, reads, k):
        self.reads = reads
        self.k = k
        self.used_edges = set()
        self.nodes = []
        self.edges = {}
        self.connections = defaultdict(list)
        self.kmers = []

    def gen_kmers(self):
        for read in self.reads:
            self.kmers.extend([read[i:self.k+i] for i in range(0, len(read)-self.k+1)])

    def create_graph(self):
        for kmer in self.kmers:
            prev_node_str = kmer[:-1]
            prev_node = Node(prev_node_str)
            next_node_str = kmer[1:]
            next_node = Node(next_node_str)
            self.nodes.extend([prev_node, next_node])
            if kmer in self.used_edges:
                self.edges[kmer].coverage += 1
            else:
                self.used_edges.add(kmer)
                self.edges[kmer] = Edge(prev_node_str, next_node_str, kmer)
                self.connections[prev_node_str].append(self.edges[kmer])

    def find_euler_path(self, v):
        connections = deepcopy(self.connections)
        string = v
        queue = [v]
        while queue:
            u = queue.pop()
            # print(u)
            for edge in connections[u]:
                if edge.coverage:
                    queue.append(edge.next_node)
                    edge.coverage -= 1
                    string += edge.next_node[-1]
        return string

    def assemble(self):
        start = self.nodes[0]
        for i in range(len(self.nodes)):
            if self.nodes[i].in_degree < start.in_degree:
                start = self.nodes[i]
        return self.find_euler_path(start.string)

        

        


G = DBGraph([true_seq], 5)
G.gen_kmers()
G.create_graph()
s1 = true_seq
s2 = G.assemble()
print(s1)
print(s2)

ACTACCCCTCTGACGAAAGTGCCGATTTTATACACTGCCAGCATCTCCGCATTTCACGACCGCGCAGAAATTGAATCATGGCATTCTTCGCCGGCAGCTATGACAGACCACAAAAGCACGCCTTTATTTCACTGATGAGATCGGTACTTCAGGCGCCCGTGGCGCCGGCATGCGTGTAGGCACGCAAGAGCGGCAAATGATGCGGCTAAACCAGGAACATGAACTCAGTTTCAAATTGTAATGTTCTAAACCTGTCTCAACTGAGCCGCTTTCGAATACGCCGAGCGGTTCCACAAGGCTTACAGCCTCTCGATTCCCTTTCCGGGACCTGCTCCTTTAGGATTACAACGTGTGAGATTTACACAAAATCCACGGTTTGTTATGTTAATTGCCTCCCGGTGCGACCTTTGTTTGACTACTTGAGCAGAGAACCCAACTCGCACACCTGTAACACAATAGGGAAGAGCGCCATTTAACCTCACATGGAGTCCCTGTGCGTGCCAAGTCTGGTAGCAGACGTGTATTGGCATACAAGACTACGTCATGAAAGTCAAAATCGGTATTTCCTTCTCCGGCTTTGACAATATACGGTAAAAACGTAGCCCCAGTACCCCAGTGGTCCAAGTTACAGACTGCGTAATAAGAATCTTTTCTGAGACCTCACGACCCATCCGTCCTTAACGAACGTGCGCCACGGACGTACTGTTATTCTCGATTCCATGATGTGTCGCTGCGGGACGGCGGATACGAAGCTCGGTTGCTGTCGCATTGGAGTCGACTGAGTGTAATAGACTGAGCAATATAGCCAGTCCAGAGACGGAAGTCTTTAGTTCCTCTCGAGTTTTGATGCTCTTGAGGCGGGACTTCCGCGGTCTCTGGTGATTCGACTAAGTCATGTGTCTCGAACTTTAGGGAGCCATGGCATGAAGTGACCAATATATGCTCGTGTCGAGTCAAGGAGCAAGGTGAATTATAGATGAATTCAGTGCTGAGAGAAGTA

In [28]:
def shared_motif(seq):
    s_seq = seq[0]
    motif = set()
    for i in range(len(s_seq)):
        for j in range(i+1,len(s_seq)+1):
            motif.add(s_seq[i:j])
    for s in seq:
        update_motif = list(motif)
        for m in update_motif:
            if m not in s:
                motif.remove(m)
    n = 0
    longest_motif = ''
    for i in motif:
        if len(i) > n:
            longest_motif = i
            n = len(i)
    return longest_motif

shared_motif([s1, s2])

'GACTACGTCAT'

## Сжатие графа  
Научитесь производить сжатие графа Де Брюина. При сжатии не забывайте склеивать подстроки на ребрах и обновлять покрытие ребер. Пересчитывайте покрытие склеиваемых ребер как взвешенное среднее, где вес соответствует длине подстроки, соответствующей ребру.